In [1]:
%pip install sdv==1.0.0

     |████████████████████████████████| 114 kB 3.9 MB/s eta 0:00:01
     |████████████████████████████████| 63 kB 3.7 MB/s eta 0:00:01
     |████████████████████████████████| 140 kB 3.6 MB/s eta 0:00:01
     |████████████████████████████████| 1.6 MB 3.0 MB/s eta 0:00:01
     |████████████████████████████████| 53 kB 2.9 MB/s eta 0:00:01
     |████████████████████████████████| 135.4 MB 7.3 MB/s eta 0:00:01    |████████████▊                   | 54.0 MB 3.7 MB/s eta 0:00:22
     |████████████████████████████████| 15.2 MB 6.4 MB/s eta 0:00:01
  Attempting uninstall: plotly
    Found existing installation: plotly 5.1.0
    Uninstalling plotly-5.1.0:
      Successfully uninstalled plotly-5.1.0
  Attempting uninstall: cloudpickle
    Found existing installation: cloudpickle 2.0.0
    Uninstalling cloudpickle-2.0.0:
      Successfully uninstalled cloudpickle-2.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the sou

In [2]:
from sdv.datasets.demo import download_demo

real_data, metadata = download_demo(
    modality='single_table',
    dataset_name='fake_hotel_guests'
)

In [35]:
print(type(real_data))

<class 'pandas.core.frame.DataFrame'>


In [6]:
real_data.shape

(500, 9)

In [3]:
metadata

{
    "METADATA_SPEC_VERSION": "SINGLE_TABLE_V1",
    "columns": {
        "guest_email": {
            "sdtype": "email",
            "pii": true
        },
        "has_rewards": {
            "sdtype": "boolean"
        },
        "room_type": {
            "sdtype": "categorical"
        },
        "amenities_fee": {
            "sdtype": "numerical",
            "computer_representation": "Float"
        },
        "checkin_date": {
            "sdtype": "datetime",
            "datetime_format": "%d %b %Y"
        },
        "checkout_date": {
            "sdtype": "datetime",
            "datetime_format": "%d %b %Y"
        },
        "room_rate": {
            "sdtype": "numerical",
            "computer_representation": "Float"
        },
        "billing_address": {
            "sdtype": "address",
            "pii": true
        },
        "credit_card_number": {
            "sdtype": "credit_card_number",
            "pii": true
        }
    },
    "primary_key": "guest_e

In [8]:
import time 

In [37]:
from sdv.single_table import CTGANSynthesizer

synthesizer = CTGANSynthesizer(metadata)

In [38]:
begin_time = time.time()
synthesizer.fit(real_data)
end_time = time.time()

In [39]:
synthesizer.get_parameters()

{'enforce_min_max_values': True,
 'enforce_rounding': True,
 'embedding_dim': 128,
 'generator_dim': (256, 256),
 'discriminator_dim': (256, 256),
 'generator_lr': 0.0002,
 'generator_decay': 1e-06,
 'discriminator_lr': 0.0002,
 'discriminator_decay': 1e-06,
 'batch_size': 500,
 'discriminator_steps': 1,
 'log_frequency': True,
 'verbose': False,
 'epochs': 300,
 'pac': 10,
 'cuda': True}

In [13]:
print(end_time - begin_time)

104.60625505447388


In [40]:
synthetic_data = synthesizer.sample(num_rows=500)
synthetic_data.head()

,guest_email,has_rewards,room_type,amenities_fee,checkin_date,checkout_date,room_rate,billing_address,credit_card_number
0,cheryl93@example.com,False,DELUXE,30.46,24 Mar 2020,24 Jan 2021,241.00,"093 Williams Lane Apt. 170\nNorth Charles, PA ...",180090934066211
1,sandersdarlene@example.org,False,BASIC,36.59,21 Aug 2020,24 Dec 2020,200.99,"PSC 5816, Box 0394\nAPO AA 74642",30083012986584
2,hsmith@example.org,False,BASIC,25.32,25 Jun 2020,04 Jan 2021,206.03,"819 Peck Curve Suite 448\nEast Johnmouth, WI 0...",2290394691481974
3,jonesernest@example.net,False,BASIC,34.73,22 Dec 2019,02 Mar 2021,274.76,"82758 Huffman Isle Suite 101\nHumphreyview, KS...",4444812351068428276
4,pstanton@example.com,False,BASIC,46.61,07 Apr 2020,11 Sep 2020,216.43,"228 Fisher Fork\nSouth Mitchell, WA 81769",4875851017747494


In [19]:
from sdv.evaluation.single_table import evaluate_quality

quality_report = evaluate_quality(
    real_data,
    synthetic_data,
    metadata
) 

Creating report: 100%|████████████████████████████| 4/4 [00:00<00:00, 45.19it/s]


Overall Quality Score: 81.02%

Properties:
Column Shapes: 82.27%
Column Pair Trends: 79.76%


In [41]:
quality_report.get_score()

0.8101637407778974

In [42]:
quality_report.get_properties()

,Property,Score
0,Column Shapes,0.822682
1,Column Pair Trends,0.797646


In [28]:
quality_report.get_details(property_name='Column Shapes')

,Column,Metric,Quality Score
0,amenities_fee,KSComplement,0.585527
1,checkin_date,KSComplement,0.806000
2,checkout_date,KSComplement,0.930564
3,room_rate,KSComplement,0.730000
4,has_rewards,TVComplement,0.990000
5,room_type,TVComplement,0.894000


In [25]:
# !pip install "plotly>=5" "ipywidgets>=7.6"

In [21]:
!jupyter labextension list

JupyterLab v3.1.7
/Users/anshu/opt/anaconda3/share/jupyter/labextensions
        jupyterlab-plotly v5.13.1 enabled OK
        @jupyter-widgets/jupyterlab-manager v3.0.0 enabled OK (python, jupyterlab_widgets)



In [36]:
quality_report.get_visualization('Column Shapes')

# https://colab.research.google.com/drive/19Bz3os18x9GTS30n4d_Tw2frUVYG1KJn?usp=sharing#scrollTo=-pQk6K3eVVPR

In [29]:
quality_report.get_properties()

,Property,Score
0,Column Shapes,0.822682
1,Column Pair Trends,0.797646


In [30]:
quality_report.get_details(property_name='Column Shapes')

,Column,Metric,Quality Score
0,amenities_fee,KSComplement,0.585527
1,checkin_date,KSComplement,0.806000
2,checkout_date,KSComplement,0.930564
3,room_rate,KSComplement,0.730000
4,has_rewards,TVComplement,0.990000
5,room_type,TVComplement,0.894000


When using this synthesizer, we can make a tradeoff between training time and data quality using the epochs parameter:
Higher epochs means that the synthesizer will train for longer, and ideally improve the data quality.

In [31]:
custom_synthesizer = CTGANSynthesizer(
    metadata,
    epochs=1000)

In [32]:
begin_time = time.time()
custom_synthesizer.fit(real_data)
end_time = time.time()

While GANs are able to model complex patterns and shapes, it is not easy to understand how they are learning -- but it is possible to modify the underlying architecture of the neural networks.

For users who are familiar with the GAN architecture, there are extra parameters you can use to tune CTGAN to your particular needs. For more details, see the CTGAN documentation.

In [33]:
end_time - begin_time

343.3822751045227

In [34]:
343/60

5.716666666666667